In [1]:
import pandas as pd 
from tensorflow import keras

In [2]:
df=pd.read_csv('Restaurant_Reviews.tsv',sep='\t')
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [3]:
import re
from nltk.stem import PorterStemmer
ps=PorterStemmer()
from nltk.corpus import stopwords
def clean_text(text):
    text=re.sub('[^a-zA-Z0-9]',' ',text)
    text=text.lower()
    text=text.split()
    text=[ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    return ' '.join(text)
df['Review']=df['Review'].apply(lambda x:clean_text(x))

In [4]:
df

,Review,Liked
0,wow love place,1
1,crust good,0
2,tasti textur nasti,0
3,stop late may bank holiday rick steve recommen...,1
4,select menu great price,1
...,...,...
995,think food flavor textur lack,0
996,appetit instantli gone,0
997,overal impress would go back,0
998,whole experi underwhelm think go ninja sushi n...,0


In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['Review'].values,df['Liked'].values,random_state=42,test_size=0.2)

In [6]:
print(x_train[:2])
print(y_train[:2])
print(x_test[:2])
print(y_test[:2])

['worst salmon sashimi' 'excel new restaur experienc frenchman']
[0 1]
['gone go' 'tri airport experi tasti food speedi friendli servic']
[1 1]


In [7]:
max_vocab_size=1500
output_sequence_length=6

In [8]:
from keras.layers import TextVectorization
text_vectorizer=TextVectorization(max_tokens=max_vocab_size,output_sequence_length=output_sequence_length)
text_vectorizer.adapt(x_train)

In [9]:
text_vectorizer(x_train[0])

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([ 75, 247, 328,   0,   0,   0], dtype=int64)>

In [10]:
from keras.layers import Embedding
embedding=Embedding(max_vocab_size,32,input_length=output_sequence_length)

In [11]:
embedding(text_vectorizer(x_train[0]))

<tf.Tensor: shape=(6, 32), dtype=float32, numpy=
array([[ 4.20552231e-02, -3.55378985e-02,  1.12605803e-02,
         1.46104433e-02,  3.95111553e-02,  3.48415487e-02,
        -3.88368256e-02, -2.79899724e-02, -3.28483582e-02,
        -2.90216934e-02, -2.16264371e-02, -1.11790523e-02,
        -2.94681322e-02, -1.56122819e-02, -6.04876131e-03,
         2.98542641e-02, -3.16721573e-02,  1.34858526e-02,
         1.30778812e-02,  3.45113128e-03, -9.09137726e-03,
         3.86345647e-02,  3.67450751e-02,  2.97434069e-02,
        -3.27052251e-02,  4.19983603e-02, -2.08476931e-03,
        -1.50724538e-02, -2.57007778e-05, -4.22563553e-02,
        -6.98281452e-03,  1.81778781e-02],
       [ 1.56018175e-02,  8.62937048e-03, -3.58318798e-02,
         2.51737498e-02, -2.46614106e-02,  4.94570173e-02,
        -3.57178822e-02, -2.10155975e-02,  3.49811204e-02,
         3.73696350e-02,  2.08926313e-02,  3.32538821e-02,
         4.53097858e-02,  3.22861336e-02, -4.46422361e-02,
        -2.75154356e-02

In [16]:
inputs=keras.layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
x=keras.layers.LSTM(16,return_sequences=True)(x)
x=keras.layers.LSTM(16,return_sequences=True)(x)
x=keras.layers.GRU(64)(x)
x=keras.layers.Dense(100,activation='relu')(x)
x=keras.layers.Dense(100,activation='relu')(x)
outputs=keras.layers.Dense(1,activation='sigmoid')(x)
model=keras.Model(inputs,outputs,name='LSTM')

In [18]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
25/25 [==============================] - 5s 45ms/step - loss: 0.0326 - accuracy: 0.9900 - val_loss: 1.6953 - val_accuracy: 0.7500
Epoch 2/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0180 - accuracy: 0.9937 - val_loss: 1.7187 - val_accuracy: 0.7400
Epoch 3/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0132 - accuracy: 0.9937 - val_loss: 1.6415 - val_accuracy: 0.7550
Epoch 4/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0096 - accuracy: 0.9975 - val_loss: 1.8020 - val_accuracy: 0.7350
Epoch 5/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0126 - accuracy: 0.9962 - val_loss: 1.6247 - val_accuracy: 0.7500
Epoch 6/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0156 - accuracy: 0.9962 - val_loss: 1.6016 - val_accuracy: 0.7500
Epoch 7/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0130 - accuracy: 0.9937 - val_loss: 2.0780 - val_accuracy: 0.7250
Epoch 8/100


25/25 [==============================] - 0s 8ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.1119 - val_accuracy: 0.7300
Epoch 59/100
25/25 [==============================] - 0s 10ms/step - loss: 0.0015 - accuracy: 0.9987 - val_loss: 3.4235 - val_accuracy: 0.7200
Epoch 60/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0026 - accuracy: 0.9987 - val_loss: 3.4981 - val_accuracy: 0.7200
Epoch 61/100
25/25 [==============================] - 0s 9ms/step - loss: 0.0034 - accuracy: 0.9987 - val_loss: 3.5175 - val_accuracy: 0.7250
Epoch 62/100
25/25 [==============================] - 0s 8ms/step - loss: 0.0035 - accuracy: 0.9987 - val_loss: 3.2867 - val_accuracy: 0.7150
Epoch 63/100
25/25 [==============================] - 0s 8ms/step - loss: 9.4983e-04 - accuracy: 1.0000 - val_loss: 3.1953 - val_accuracy: 0.7200
Epoch 64/100
25/25 [==============================] - 0s 8ms/step - loss: 9.7043e-04 - accuracy: 1.0000 - val_loss: 3.5678 - val_accuracy: 0.7250
Epoch 65/1

In [19]:
model.evaluate(x_train,y_train)

25/25 [==============================] - 1s 2ms/step - loss: 7.9137e-06 - accuracy: 1.0000


[7.913707122497726e-06, 1.0]

In [20]:
model.evaluate(x_test,y_test)

7/7 [==============================] - 0s 3ms/step - loss: 9.9612 - accuracy: 0.7200


[9.961248397827148, 0.7200000286102295]